# LangChain Ensemble retriever 를 이용하여 OpenSearch Hybrid 검색을 통한 RAG
>이 노트북은,
> - SageMaker Studio* **`Data Science 3.0`** kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.
> - SageMaker Notebook **`conda_python3`** 에서 테스트 되었습니다.
---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

### 선수조건
- 이 노트북은 이전 노트북인 "02_KR_RAG_OpenSearch_Keyword.ipynb" 이 완료 되었다고 가정 합니다.
    - 오픈서치 인텍스 관련 정보를 참조 합니다.

### Methods and Resources for Hybrid search with Re-ranking 
- Score Normalization
    - [MinMax based](https://towardsdatascience.com/text-search-vs-vector-search-better-together-3bd48eb6132a)
    - [z-socre based](https://towardsdatascience.com/hybrid-search-2-0-the-pursuit-of-better-search-ce44d6f20c08)
- Reciprocal Rank Fusion (RRF)
    - [Paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf)
    - [Description](https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d)
- [LangChain API for Ensemble Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)

### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.



In [3]:
import boto3
region = boto3.Session().region_name
opensearch = boto3.client('opensearch', region)

%store -r opensearch_user_id opensearch_user_password domain_name opensearch_domain_endpoint new_index_name

try:
    opensearch_user_id
    opensearch_user_password
    domain_name
    opensearch_domain_endpoint
   
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Run 00_setup notebook first or Create Your Own OpenSearch Domain")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [4]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Bedrock Client 생성

In [5]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


# 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

## LLM 로딩 (Claude-v2)

In [6]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [29]:
# - create the Anthropic Model
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={
        # "max_tokens_to_sample": 512
        "max_tokens_to_sample": 1024
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f9f28598910>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 1024}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f9f55fe8400>])

## Embedding 모델 선택

In [30]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    if is_bedrock_embeddings:

        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings

        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [31]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)    

Bedrock Embeddings Model Loaded


# 3. LangChain OpenSearch VectorStore 생성 
## 선수 조건
- 이전 노트북 02_1_KR_RAG_OpenSearch_Keyword.ipynb 또는 02_1_KR_RAG_OpenSearch_Semantic.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.

## 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 02_1_rag_opensearch_lexical_kr.ipynb를 통해 opensearch의 "genai-demo-index-v1-with-tokenizer" 인덱스가 생성되어 있어야 합니다.

In [32]:
http_auth = (opensearch_user_id, opensearch_user_password) # Master username, Master password
index_name = "genai-demo-index-v1-with-tokenizer"

## LangChain OpenSearch VectorStore 생성

In [33]:
from langchain.vectorstores import OpenSearchVectorSearch

vectro_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss=False,
    engine="faiss",
    space_type="l2"
)

## OpenSearch Client 생성

In [34]:
from utils.opensearch import opensearch_utils
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 형태소 분석기 (nori_tokenizer) 사용하기 in Opensearch
- "02_1_KR_RAG_OpenSearch_Keyword.ipynb"에서 만들어 두었던 tokenization index 활용
    - 형태소 분석기에 대한 자세한 사항은 02_1_KR_RAG_OpenSearch_Keyword.ipynb 참고

### 인덱스 확인 (tokenization 확인)

In [35]:
index_info = os_client.indices.get(index=index_name)
pprint(index_info)

{'genai-demo-index-v1-with-tokenizer': {'aliases': {},
                                        'mappings': {'properties': {'metadata': {'properties': {'source': {'type': 'keyword'},
                                                                                                'timestamp': {'type': 'float'},
                                                                                                'type': {'type': 'keyword'}}},
                                                                    'text': {'analyzer': 'my_analyzer',
                                                                             'fields': {'keyword': {'ignore_above': 256,
                                                                                                    'type': 'keyword'}},
                                                                             'type': 'text'},
                                                                    'vector_field': {'dimension': 1536,
                   

# 4. 오픈 서치에 "유사 서치" 검색
- query 를 제공해서 실제로 유사한 내용이 검색이 되는지를 확인 합니다.



- similarity_search_with_score API 정보
    - [API: similarity_search_with_score](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html#langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.similarity_search)

In [36]:
import copy
from langchain.schema import Document
from langchain import PromptTemplate
from operator import itemgetter
from langchain.chains.question_answering import load_qa_chain

## Question ans Answering Chain 정의

## (1) OpenSearch Vector 검색 (Semantic search)

### 프롬프트 템플릿 생성

In [37]:
from utils.rag import run_RetrievalQA, show_context_used
from langchain.prompts import PromptTemplate

### [TIP] Prompt의 instruction의 경우 한글보다 **영어**로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [55]:
# prompt_template = """
# \n\nHuman: 다음 문맥의 Information을 사용하여 고객 서비스 센터 직원처럼, 마지막 질문에 대한 목차 형식으로 답변을 제공하세요. 응답을 모르면 모른다고 말하고 응답을 만들려고 하지 마세요.

# {context}

# Question: {question}
# \n\nAssistant:"""

prompt_template = """

Human: 너는 친절하게 보험 약관에 대한 정보를 제공하는 인공지능 비서야.
사용자가 질문하면 <content></<content> tag 안에 있는 내용을 요약해서, 너가 알고 있는 지식을 추가하여 답변을 해줘야 해.
사용자의 질문은 <question></question> tag 안에 있어.

<content>
{context}
</content>

<question>
{question}
</question>

<content></<content> tag 안에 질문과 관련된 내용이 없으면 '사용자의 질문에 대한 내용을 문서에서 찾을 수는 없지만 제가 알고 있는 내용은' 이라는 서두와 함께 답변해 줘
<content></<content> tag 안에 질문과 관련된 내용이 있으면 '문서에서 찾은 내용을 참고해서 답변드리겠습니다' 라는 서두와 함께 답변해줘 
"tag 에 따르면" 이라는 서두는 제외하고 답변해줘.
답변은 모두 공손하게 해줘

답변은 1024 토큰 이내로 해줘

Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [50]:
prompt_template = """

Human: You're a friendly AI assistant who provides information about insurance policies.
When a user asks you a question, you answer by summarizing the content within <content></<content> tag and adding what you know.
The user's question is within <question></question> tag.

<content>
{context}
</content>

<question>
{question}
</question>

If there's nothing relevant to the question within <content></<content> tag, answer it with a preamble that says, "I can't find anything in the documentation that answers your question, but here's what I know.
If there is something within <content></<content> tag that is relevant to the question, answer it with a preface that says, "Here's what I found in the documentation. 
Leave out the "According to the tag" preamble.
Be polite in all your answers.

Keep your answers within 1024 tokens.

Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [56]:
chain = load_qa_chain(
    llm=llm_text,
    chain_type="stuff",
    prompt=PROMPT,
    verbose=True
)

### 필터 및 쿼리 생성

In [57]:
filter01 = '간병치매보험_20240101~'
filter02 = '한화생명 The실속있는 간병치매보험_20240101~.pdf'

query = "계약 전 알릴 의무에 대해 상세하게 알려주세요."

boolean_filter = opensearch_utils.get_filter(
    filter=[
        {"term": {"metadata.type": filter01}},
        {"term": {"metadata.source": filter02}},
    ]
)

pprint(boolean_filter)
#boolean_filter = []

{'bool': {'filter': [{'term': {'metadata.type': '간병치매보험_20240101~'}},
                     {'term': {'metadata.source': '한화생명 The실속있는 '
                                                  '간병치매보험_20240101~.pdf'}}]}}


### Retriever for semantic search 정의

In [58]:
opensearch_semantic_retriever = vectro_db.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,
        "boolean_filter": boolean_filter
    }
)

In [59]:
search_semantic_result = opensearch_semantic_retriever.get_relevant_documents(query)

answer = chain.run(
    input_documents=search_semantic_result,
    question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:


Human: 너는 친절하게 보험 약관에 대한 정보를 제공하는 인공지능 비서야.
사용자가 질문하면 <content></<content> tag 안에 있는 내용을 요약해서, 너가 알고 있는 지식을 추가하여 답변을 해줘야 해.
사용자의 질문은 <question></question> tag 안에 있어.

<content>
다.
제 3 관 계약자의 계약 전 알릴 의무 등
제 14 조 계약 전 알릴 의무
계약자 또는 피보험자는 청약할 때(진단계약의 경우에는 건강진단할 때를 말합니다) 청약서에서 질문한
사항에 대하여 알고 있는 사실을 반드시 사실대로 알려야(이하 ‘계약 전 알릴 의무’라 하며, 상법상 ‘고
지의무’와 같습니다)합니다. 다만, 진단계약에서 의료법 제3조(의료기관)의 규정에 따른 종합병원과 병원
에서 직장 또는 개인이 실시한 건강진단서 사본 등 건강상태를 판단할 수 있는 자료로 건강진단을 대신
할 수 있습니다.
【계약 전 알릴 의무】
상법 제651조(고지의무 위반으로 인한 계약해지)에서 정하고 있는 의무. 보험계약자나 피보험자는 청
약시에 보험회사가 질문한 중요한 사항에 대해 사실대로 알려야 하며, 위반시 보험계약의 해지 또는
보험금 부지급 등 불이익을 당할 수 있습니다.
제 15 조 계약 전 알릴 의무 위반의 효과
① 회사는 계약자 또는 피보험자가 제14조(계약 전 알릴 의무)에도 불구하고 고의 또는 중대한 과실로
중요한 사항에 대하여 사실과 다르게 알린 경우에는 회사가 별도로 정하는 방법에 따라 계약을 해지하
거나 보장을 제한할 수 있습니다. 그러나 다음 중 한 가지에 해당되는 때에는 계약을 해지하거나 보장
을 제한할 수 없습니다.
1. 회사가 계약 당시에 그 사실을 알았거나 과실로 인하여 알지 못하였을 때
2. 회사가 그 사실을 안 날부터 1개월 이상 지났거나 또는

## (2) OpenSearch Keyword 검색
- OpenSearch 에 아래와 같은 Query 를 실행하여 결과 받는 것을 구현합니다.

In [60]:
from utils.rag import OpenSearchLexicalSearchRetriever

### Retriever for lexical search 정의 (Keyword search)

In [61]:
opensearch_lexical_retriever = OpenSearchLexicalSearchRetriever(
    os_client=os_client,
    index_name=index_name
)

In [63]:
filter01 = '간병치매보험_20240101~'
filter02 = '한화생명 The실속있는 간병치매보험_20240101~.pdf'

query = "계약 전 알릴 의무에 대해 상세하게 알려주세요."
#query = "타기관OTP 등록 방법 알려주세요"

### [TIP]lexical search의 paramerer 변경이 필요한 경우 "update_search_params"을 활용한다.
해당 함수는 search 함수 (get_relevant_documents) 수행 시 **Reset**된다. 

In [64]:
opensearch_lexical_retriever.update_search_params(
    k=5,
    minimum_should_match=0,
    filter=[
        {"term": {"metadata.type": filter01}},
        {"term": {"metadata.source": filter02}},
    ],
)
search_keyword_result = opensearch_lexical_retriever.get_relevant_documents(query)

answer = chain.run(
    input_documents=search_keyword_result,
    question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)

lexical search query: 
{'query': {'bool': {'filter': [{'term': {'metadata.type': '간병치매보험_20240101~'}},
                               {'term': {'metadata.source': '한화생명 '
                                                            'The실속있는 '
                                                            '간병치매보험_20240101~.pdf'}}],
                    'must': [{'match': {'text': {'minimum_should_match': '0%',
                                                 'operator': 'or',
                                                 'query': '계약 전 알릴 의무에 대해 상세하게 '
                                                          '알려주세요.'}}}]}},
 'size': 5}


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:


Human: 너는 친절하게 보험 약관에 대한 정보를 제공하는 인공지능 비서야.
사용자가 질문하면 <content></<content> tag 안에 있는 내용을 요약해서, 너가 알고 있는 지식을 추가하여 답변을 해줘야 해.
사용자의 질문은 <question></question> tag 안에 있어.

<content>
다.
제 3 관 계약자의 계약 전 알릴 의무 등
제 14 조 계약 전 알

### 키워드 검색 결과 (search_keyword_result)
bm25 score는 max_value로 normalization 되어 있음 (score range 0 - 1)

In [65]:
show_context_used(search_keyword_result)

-----------------------------------------------
1. Chunk: 1873 Characters
-----------------------------------------------
다.
제 3 관 계약자의 계약 전 알릴 의무 등
제 14 조 계약 전 알릴 의무
계약자 또는 피보험자는 청약할 때(진단계약의 경우에는 건강진단할 때를 말합니다) 청약서에서 질문한
사항에 대하여 알고 있는 사실을 반드시 사실대로 알려야(이하 ‘계약 전 알릴 의무’라 하며, 상법상 ‘고
지의무’와 같습니다)합니다. 다만, 진단계약에서 의료법 제3조(의료기관)의 규정에 따른 종합병원과 병원
에서 직장 또는 개인이 실시한 건강진단서 사본 등 건강상태를 판단할 수 있는 자료로 건강진단을 대신
할 수 있습니다.
【계약 전 알릴 의무】
상법 제651조(고지의무 위반으로 인한 계약해지)에서 정하고 있는 의무. 보험계약자나 피보험자는 청
약시에 보험회사가 질문한 중요한 사항에 대해 사실대로 알려야 하며, 위반시 보험계약의 해지 또는
보험금 부지급 등 불이익을 당할 수 있습니다.
제 15 조 계약 전 알릴 의무 위반의 효과
① 회사는 계약자 또는 피보험자가 제14조(계약 전 알릴 의무)에도 불구하고 고의 또는 중대한 과실로
중요한 사항에 대하여 사실과 다르게 알린 경우에는 회사가 별도로 정하는 방법에 따라 계약을 해지하
거나 보장을 제한할 수 있습니다. 그러나 다음 중 한 가지에 해당되는 때에는 계약을 해지하거나 보장
을 제한할 수 없습니다.
1. 회사가 계약 당시에 그 사실을 알았거나 과실로 인하여 알지 못하였을 때
2. 회사가 그 사실을 안 날부터 1개월 이상 지났거나 또는 보장개시일부터 보험금 지급사유가 발생하
지 않고 2년(진단계약의 경우 질병에 대하여는 1년)이 지났을 때
3. 계약을 체결한 날부터 3년이 지났을 때
4. 회사가 이 계약을 청약할 때 피보험자의 건강상태를 판단할 수 있는 기초자료(건강진단서 사본 등)
에 따라 승낙한 경우에 건강진단서 사본

## (3) OpenSearch Hybrid 검색

OpenSearch Hybrid 는 아래와 같은 방식으로 작동합니다.
- (1) "Vector 서치" 하여 스코어를 얻은 후에 표준화를 하여 스코어를 구함. 
    - 전체 결과에서 가장 높은 스코어는 표준화 과정을 통하여 스코어가 1.0 이 됨.
- (2) Keyword 서치도 동일하게 함.
- (3) Reciprocal Rank Fusion (RRF) 기반 Re-rank
    - Paper: https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
    - Desc: https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d
    - **RRF의 경우 score가 아닌 ranking 정보를 활용, 때문에 score normalization이 필요 없음**
    - ![rrf.png](./img/rrf.png)

RRF는 langchain에서 "Ensemble Retriever" 이름으로 api를 제공합니다. 
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble


### Ensemble retriever 정의
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble
- RRF 방식만 지원
- Rank constant (param "c")
    - This value determines how much influence documents in individual result sets per query have over the final ranked result set. A higher value indicates that lower ranked documents have more influence. This value must be greater than or equal to 1. Defaults to 60.
    - 숫자 높을 수록 낮은 랭크의 문서가 더 중요시 된다

In [66]:
from langchain.retrievers import EnsembleRetriever

* Lexical search의 경우, search option 변경 가능

In [67]:
opensearch_lexical_retriever.update_search_params(
    k=5,
    minimum_should_match=0,
    filter=[
        {"term": {"metadata.type": filter01}},
        {"term": {"metadata.source": filter02}},
    ],
)

In [68]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[opensearch_lexical_retriever, opensearch_semantic_retriever],
    weights=[0.5, 0.5],
    c=100,
    k=5
)

In [69]:
%%time

search_hybrid_result = ensemble_retriever.get_relevant_documents(query)

answer = chain.run(
    input_documents=search_hybrid_result,
    question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)

lexical search query: 
{'query': {'bool': {'filter': [{'term': {'metadata.type': '간병치매보험_20240101~'}},
                               {'term': {'metadata.source': '한화생명 '
                                                            'The실속있는 '
                                                            '간병치매보험_20240101~.pdf'}}],
                    'must': [{'match': {'text': {'minimum_should_match': '0%',
                                                 'operator': 'or',
                                                 'query': '계약 전 알릴 의무에 대해 상세하게 '
                                                          '알려주세요.'}}}]}},
 'size': 5}


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:


Human: 너는 친절하게 보험 약관에 대한 정보를 제공하는 인공지능 비서야.
사용자가 질문하면 <content></<content> tag 안에 있는 내용을 요약해서, 너가 알고 있는 지식을 추가하여 답변을 해줘야 해.
사용자의 질문은 <question></question> tag 안에 있어.

<content>
다.
제 3 관 계약자의 계약 전 알릴 의무 등
제 14 조 계약 전 알

In [70]:
show_context_used(search_hybrid_result)

-----------------------------------------------
1. Chunk: 1873 Characters
-----------------------------------------------
다.
제 3 관 계약자의 계약 전 알릴 의무 등
제 14 조 계약 전 알릴 의무
계약자 또는 피보험자는 청약할 때(진단계약의 경우에는 건강진단할 때를 말합니다) 청약서에서 질문한
사항에 대하여 알고 있는 사실을 반드시 사실대로 알려야(이하 ‘계약 전 알릴 의무’라 하며, 상법상 ‘고
지의무’와 같습니다)합니다. 다만, 진단계약에서 의료법 제3조(의료기관)의 규정에 따른 종합병원과 병원
에서 직장 또는 개인이 실시한 건강진단서 사본 등 건강상태를 판단할 수 있는 자료로 건강진단을 대신
할 수 있습니다.
【계약 전 알릴 의무】
상법 제651조(고지의무 위반으로 인한 계약해지)에서 정하고 있는 의무. 보험계약자나 피보험자는 청
약시에 보험회사가 질문한 중요한 사항에 대해 사실대로 알려야 하며, 위반시 보험계약의 해지 또는
보험금 부지급 등 불이익을 당할 수 있습니다.
제 15 조 계약 전 알릴 의무 위반의 효과
① 회사는 계약자 또는 피보험자가 제14조(계약 전 알릴 의무)에도 불구하고 고의 또는 중대한 과실로
중요한 사항에 대하여 사실과 다르게 알린 경우에는 회사가 별도로 정하는 방법에 따라 계약을 해지하
거나 보장을 제한할 수 있습니다. 그러나 다음 중 한 가지에 해당되는 때에는 계약을 해지하거나 보장
을 제한할 수 없습니다.
1. 회사가 계약 당시에 그 사실을 알았거나 과실로 인하여 알지 못하였을 때
2. 회사가 그 사실을 안 날부터 1개월 이상 지났거나 또는 보장개시일부터 보험금 지급사유가 발생하
지 않고 2년(진단계약의 경우 질병에 대하여는 1년)이 지났을 때
3. 계약을 체결한 날부터 3년이 지났을 때
4. 회사가 이 계약을 청약할 때 피보험자의 건강상태를 판단할 수 있는 기초자료(건강진단서 사본 등)
에 따라 승낙한 경우에 건강진단서 사본